In [1]:
import os
import re
import csv
import sys
import ast 
import json
import time
import urllib
import random
import spotipy
import requests
import datetime
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# spotifyAPI存取權限
client_id = '1a23ed1ca359485ab81d524206dba0f6'
client_secret = 'af0e14fdb19a49dfb19ef0c1ea1d42ea'
username = 'qnwv65t11cplaz4dikhl4mjgi'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# 檔案名稱所有非法字元都轉義：https://www.itread01.com/content/1549125182.html
def validateTitle(title):
    rstr = r"[\/\\\:\*\?\"\<\>\|]" # '/ \ : * ? " < > |'
    new_title = re.sub(rstr, " ", title)  # 替換為空白
    return new_title

In [4]:
# 取得歌手的所有專輯
def show_artist_albums(artist_id):
    albums = []
    # album_type有四種：album、single、appears_on、compilation (include_groups=album,single)
    results = sp.artist_albums(artist_id, album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    # 移除重複的album
    seen = set()
    # 按照專輯的名稱排序
    albums.sort(key=lambda album: album['name'].lower())
    list_album_name = []
    list_album_id = []
    list_album_uri = []
    list_album_href = []
    for album in albums:
        album_name = album['name']
        album_id = album['id']
        album_uri = album['uri']
        album_href = 'https://open.spotify.com/album/' + album_id
        if album_name not in seen:
            #print(album_name)
            seen.add(album_name)
            list_album_name.append(album_name)
            list_album_id.append(album_id)
            list_album_uri.append(album_uri)
            list_album_href.append(album_href)
    df_albums = pd.DataFrame(list(zip(list_album_name,list_album_id,list_album_uri,list_album_href)), columns =['Name','Id','Uri','Href'])
    df_albums.drop_duplicates(subset='Name', keep='first', inplace=True)
    df_albums.reset_index(drop=True, inplace=True)
    return df_albums

In [7]:
# 每位歌手抓取專輯資料並輸出csv(df_albums_+ artist_name +.csv)
empty_album = [] #沒有專輯的歌手
exist_album = [] #有專輯的歌手

df_id_genres = pd.read_csv(r'csv/20200303/artist_list_ID+Genres_select.csv')
for i in range(len(df_id_genres)):
    artist_name = df_id_genres['Artist'][i]
    artist_id = df_id_genres['Id'][i]
    
    df_albums = show_artist_albums(artist_id)
    # 歌手沒有專輯data
    if df_albums.empty == True:
        empty_album.append(artist_name) #沒有專輯data的歌手名稱放入empty_album列表中
        print(str(i) + ' | ' + str(artist_name) + ' | ' + 'N')
    # 歌手有專輯data
    if df_albums.empty == False:
        exist_album.append(artist_name) #有專輯data的歌手名稱放入exist_album列表中
        artist_name = validateTitle(artist_name) #檔名去除無效字元
        df_albums.to_csv('csv/20200303/albums/df_albums_'+ artist_name +'.csv',index=0,encoding="utf_8_sig")
        print(str(i) + ' | ' + str(artist_name) + ' | ' + 'Y')

print('DONE')

0 | 5 Seconds of Summer | Y
1 | Adam Lambert | Y
2 | Adele | Y
3 | Alec Benjamin | Y
4 | Ariana Grande | Y
5 | Ashley Tisdale | Y
6 | Avril Lavigne | Y
7 | Backstreet Boys | Y
8 | Bad Bunny | Y
9 | Bazzi | Y
10 | Beyoncé | Y
11 | Billie Eilish | Y
12 | Bruno Mars | Y
13 | Calvin Harris | Y
14 | Camila Cabello | Y
15 | Charlie Puth | Y
16 | Chris Brown | Y
17 | Christina Aguilera | Y
18 | Christina Grimmie | Y
19 | Christina Perri | Y
20 | Coldplay | Y
21 | David Guetta | Y
22 | Drake | Y
23 | Ed Sheeran | Y
24 | Ellie Goulding | Y
25 | Elvis Presley | Y
26 | Enrique Iglesias | Y
27 | Grace VanderWaal | Y
28 | Gwen Stefani | Y
29 | Hailee Steinfeld | Y
30 | Halsey | Y
31 | Jennifer Lopez | Y
32 | John Legend | Y
33 | Justin Bieber | Y
34 | Justin Timberlake | Y
35 | Katy Perry | Y
36 | Khalid | Y
37 | Lady Gaga | Y
38 | Lana Del Rey | Y
39 | Lauv | Y
40 | Madonna | Y
41 | Maroon 5 | Y
42 | Miley Cyrus | Y
43 | Niall Horan | Y
44 | OneRepublic | Y
45 | Ozuna | Y
46 | Rihanna | Y
47 | Sab